# OPEN PROBLEM: Lightcurve analysis

We select a subsample of observations from the internal CTAO SDC of the source MRK 421. This is a time-varying source and the aim of this tutorial is to build a lightcurve with the available data.

1) Let's start with the basic imports:

In [1]:
import matplotlib.pyplot as plt

import numpy as np
import astropy.units as u
from astropy.coordinates import Angle, SkyCoord
from astropy.time import Time
from regions import CircleSkyRegion
from gammapy.data import (
    DataStore,
    Observation,
    Observations,
)
from gammapy.datasets import MapDataset, Datasets, SpectrumDataset
from gammapy.estimators import LightCurveEstimator
from gammapy.estimators.utils import get_rebinned_axis
from gammapy.makers import (
    MapDatasetMaker,
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    FoVBackgroundMaker
)
from gammapy.maps import MapAxis, WcsGeom, RegionGeom
from gammapy.modeling.models import (
    Models,
    PowerLawSpectralModel,
    SkyModel,
)

2) Select all the observations from the index flies, using the `DataStore` class, and pick only observations enclosed into a search cone of 5 degrees around the given pointing position:

In [3]:
data_store = 

3) Define the target position (ra, dec = 166.113808°, 38.2088329°) and the pointing direction (consider an offset of 0.5° with respect to the target position):

In [4]:
#source coordinates
target = 

#pointing coordinates
pointing = 

4) Create the dictionary for the observation selection as in the 3D analysis notebook, and pick them with the `select_observation` function of `DataStore`:

In [5]:
selection = 

selected_obs_table = 

5) Take a subsample of the first 50 observations.
Use them with the `get_observations` function:

In [ ]:
selected_obs_table =  

observations = 

print(f"Selected observations: {len(observations)}")
print(f"\n\nLet's have a look at the first observation \n\n: {observations[0]}")

6) Create the exclusion mask around the source region, as in the 3D analysis notebook:

In [ ]:
exclusion_geom = 

exclusion_src1 = 

exclusion_mask = 

exclusion_mask.plot()
plt.show()

7) Define the reconstructed and true energy axes (with `MapAxis`) and the geometry (with `WcsGeom.create`), as in the 3D analysis notebook:

In [8]:
energy_axis = 

energy_axis_true = 

geom = 

8) Create the MapDataset:

In [9]:
stacked = 

9) Set the safe ranges with `MapDatasetMaker`, `SafeMaksMaker` and `FoVBackgroundMaker`:

In [10]:
offset_max = 
maker = 
maker_safe_mask = 

maker_fov = 

10) Loop over the observations to stack the datasets:

In [ ]:
for i, obs in enumerate(observations):
    # First a cutout of the target map is produced
    cutout = 
    
    # A MapDataset is filled in this cutout geometry
    dataset = 
    
    # The data quality cut is applied
    dataset = 
    
    # fit background model
    dataset = 
    
    print(
        f"{i} of {len(observations)}, Background norm obs {obs.obs_id}: {dataset.background_model.spectral_model.norm.value:.2f}"
    )
    # The resulting dataset cutout is stacked onto the final one
    stacked.stack...

print(stacked)

11) Make a counts map:

# Make a lightcurve

12) To build a lightcurve, you should firstly define the time bin where the lightcurve has to be estimated. We give you the hints on how to create them but, please, for more details look at the Gammapy tutorial:

In [13]:
# define the time intervals, splitting the full observation time range in bins of 2 hr:
t0 = observations[0].tstart
tstop = observations[-1].tstop
duration =  #choose a good duration with units
n_time_bins = int( ((tstop-t0)).to("s") / duration) + 10
times = t0 + np.arange(n_time_bins) * duration
time_intervals = [Time([tstart, tstop]) for tstart, tstop in zip(times[:-1], times[1:])]

13) Filter all the observations that satisfy the temporal requirements using the `select_time` function:

In [ ]:
short_observations = 

# check that observations have been filtered
print(f"Number of observations after time filtering: {len(short_observations)}\n")
print(short_observations[1].gti)

14) Create the energy axes (reco and true) and perform an 1D spectral analysis of the source. In this case, we will adopt the reflected region method:

In [29]:
# Target definition
energy_axis = 
energy_axis_true = 

on_region_radius = 
on_region = 

geom =  

15) Set the safe ranges with the `SafeMaskMaker` class, produce the reflected regions with `ReflectedRegionsBackgroundMaker` and make the `SpectrumDataset`:

In [30]:
dataset_maker = 

bkg_maker =

safe_mask_masker = 

16) Create the `Datasets` for each observation and stack them:

In [31]:
datasets = Datasets()

dataset_empty = 

for obs in short_observations:
    dataset = 

    dataset_on_off = 
    dataset_on_off = 
    datasets.append(dataset_on_off)

16) Define a spectral model with `PowerLawSpectralModel`. First fit the spectral model on the global stacked dataset, then use the model as the fixed best-one.

In [32]:
spectral_model = 
spectral_model.parameters... 

sky_model = 

17) Associate the sky-model to the datasets:

In [33]:
datasets.models = sky_model

18) Make the lightcurve in two energy bands. Is the source spectrally varying?
    Make the lightcurve over the time bins defined above for `MRK 421`, using the Gammapy class `LightCurveEstimator`:

In [20]:
lc_maker_1d = 

lc_maker_1d.norm.scan_max = 10

lc_1d = 

19) Plot the lightcurve:

20) Try to correct for the EBL (assuming the known redshift of MRK 421).